# West Midlands Street Crime Analysis

All Street Crime 04-2020 to 03-2023

**1. pyspark Installation**

In [ ]:
# install PySpark 
#http://spark.apache.org/docs/latest/api/python/index.html

!pip install pyspark==3.2

**2.Spark session & SQLContext**

In [ ]:
import warnings
warnings.filterwarnings("ignore") 

# start spark session and configuration
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
spark = SparkSession.builder.master("local[2]").appName("dat").getOrCreate()
sc = spark.sparkContext

#create an instance of SQLContext
sqlContext = SQLContext(spark)

**3. Import Main Libraries**

In [ ]:
# Import libraries and other functions
from io import StringIO
from collections import namedtuple
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *

import csv
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np


**4. Create Schema**

In [ ]:
myschema = StructType(
    [StructField("Crime ID", StringType(), True),
     StructField("Month", DateType(), True),
     StructField("Reported by", StringType(), True),
     StructField("Falls within", StringType(), True),
     StructField("Longitude", FloatType(), True),
     StructField("Latitude", FloatType(), True),
     StructField("Location", StringType(), True),
     StructField("LSOA code", StringType(), True),
     StructField("LSOA name", StringType(), True),
     StructField("Crime type", StringType(), True),
     StructField("Last outcome category", StringType(), True),
     StructField("Context", StringType(), True)])  

**5.Read West Midlands dataset**

In [ ]:
#create a dataFrame directly from City of west midlands Street Crime csv file 
#https://data.police.uk/data/
# Read the dataset with custom schema
WestMidlandsCrime = spark.read\
    .option("header", "true")\
    .option("delimiter", ",")\
    .option("inferSchema", "false")\
    .schema(myschema)\
    .csv("/kaggle/input/west-midlands-crime/*street.csv")

**6.Check for Null Values**

In [ ]:
from pyspark.sql.functions import col, sum

# Check for null values in each column
null_counts = WestMidlandsCrime.select([sum(col(c).isNull().cast("int")).alias(c) for c in WestMidlandsCrime.columns])

# Display the null counts
null_counts.toPandas()

**7. Analyse null values for Crime ID and Last outcome category**

In [ ]:
from pyspark.sql.functions import col

# Filter the DataFrame for rows where Crime ID and Last outcome category are null
null_data = WestMidlandsCrime.filter(col("Crime ID").isNull() & col("Last outcome category").isNull())

# Get distinct crime types from the filtered data
distinct_crime_types = null_data.select("Crime type").distinct().collect()

# Extract the crime types as a list
crime_types = [row["Crime type"] for row in distinct_crime_types]

# Print the crime types with null values in Crime ID and Last outcome category
print(f"The crime type with null values in 'Crime ID' and 'Last outcome category' is: {', '.join(crime_types)}")


**8. Analyze Null Values for LSOA_code & LSOA_name**

In [ ]:
# Assuming your DataFrame is named "WestMidlandsCrime"
missing_lsoa_records = WestMidlandsCrime.filter(WestMidlandsCrime['LSOA_name'].isNull())

# Display the records with missing LSOA names and their corresponding locations
missing_lsoa_records.select('Location','LSOA_code', 'LSOA_name').show(truncate=False)


**9. Print Schema**

In [ ]:
WestMidlandsCrime.printSchema()

**10. Rename all Necessary columns**

In [ ]:
# tidy up the column names

WestMidlandsCrime = WestMidlandsCrime.withColumnRenamed('Crime ID', 'Crime_ID')
WestMidlandsCrime = WestMidlandsCrime.withColumnRenamed('Reported by', 'Reported_by')
WestMidlandsCrime = WestMidlandsCrime.withColumnRenamed('Falls within', 'Falls_within')
WestMidlandsCrime = WestMidlandsCrime.withColumnRenamed('LSOA code', 'LSOA_code')
WestMidlandsCrime = WestMidlandsCrime.withColumnRenamed('LSOA name', 'LSOA_name')
WestMidlandsCrime = WestMidlandsCrime.withColumnRenamed('Crime type', 'Crime_type')
WestMidlandsCrime = WestMidlandsCrime.withColumnRenamed('Last outcome category', 'Last_outcome_category')

**11.Remove context and crime id**

Context has all rows with null values and crime id has null values for anti social behaviour and not a necessary coloumn as its hexademical and does not required for any analysis.


In [ ]:
WestMidlandsCrime = WestMidlandsCrime.drop("Context", "Crime_ID")

In [ ]:
WestMidlandsCrime.printSchema()

**12. Check the data looks good for Analyses**

In [ ]:
WestMidlandsCrime.take(3)

**13. Display Total Count**

In [ ]:
print(WestMidlandsCrime.count())

**14. Data Analysis**

**Create Temperory view**

In [ ]:
WestMidlandsCrime.createTempView("wmdata")

**Question 1 : What crime categories does the Police data use?**

In [ ]:
WestMidlandsCrimeType = spark.sql("select distinct Crime_type from wmdata")

In [ ]:
WestMidlandsCrimeType.show()

In [ ]:
WestMidlandsCrimeType.toPandas()

**Question 2 : How is Last Outcome Categories distributed**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the crime data into a pandas DataFrame
crime_df = spark.sql("SELECT * FROM wmdata").toPandas()

# Analyze the distribution of outcome categories
outcome_counts = crime_df['Last_outcome_category'].value_counts()

# Plot the distribution of outcome categories
plt.figure(figsize=(12, 6))
outcome_counts.plot(kind='bar')
plt.xlabel('Outcome Category')
plt.ylabel('Count')
plt.title('Distribution of Last Outcome Categories')
plt.show()


**Geographical Distribution of Last Outcome Categories**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming your DataFrame is named "WestMidlandsCrime"
crime_df = WestMidlandsCrime.toPandas()

# Select the columns of interest for geographical analysis
geo_df = crime_df[['Longitude', 'Latitude', 'Last_outcome_category']]

# Create a scatter plot to visualize the geographical distribution
plt.figure(figsize=(10, 8))
sns.scatterplot(x='Longitude', y='Latitude', hue='Last_outcome_category', data=geo_df)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Geographical Distribution of Outcome Categories')
plt.legend()
plt.show()

**Question 3 : Have the same categories been used consistently?**

In [ ]:
WMmonthcat = spark.sql("select distinct Crime_type, EXTRACT(YEAR FROM Month) as Year from wmdata group by Crime_type, Year order by Year")

In [ ]:
WMmonthcat.show()

In [ ]:
WMmonthcat.toPandas()

**Question 4 : What are the types of crimes recorded in West Midlands?**

In [ ]:
wm_top= spark.sql("SELECT Crime_type, COUNT(*) AS num_crimes FROM wmdata GROUP BY Crime_type")

In [ ]:
wm_top.show()

In [ ]:
wm_top.toPandas()

**Graphical view of crime type vs count**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming you have a PySpark DataFrame named "wm_top" with columns "Month" and "CrimeNumbers"

# Convert PySpark DataFrame to Pandas DataFrame
wm_top_pd = wm_top.toPandas()

# Create the plot using Pandas DataFrame
plt.figure(figsize=(10, 6))
plt.plot(wm_top_pd['Crime_type'], wm_top_pd['num_crimes'], marker='.', linestyle='-', linewidth=1, color='red')
plt.title('Crime Numbers Vs Crime_type')
plt.xticks(rotation=90)
plt.xlabel('Crime_type')
plt.ylabel('Crime Numbers')
plt.show()

**Question 5 : Are levels of violent crime constant, increasing, or decreasing?**

In [ ]:
filtereddata = spark.sql("select Month, count(Crime_type) as CrimeNumbers FROM wmdata where Crime_type='Violence and sexual offences' and Month >= '2020-08-01' and Month <= '2023-03-31' group by Month order by Month")

In [ ]:
filpanda = filtereddata.toPandas()
filpanda

In [ ]:
filpanda.Month = filpanda.Month.astype('datetime64[ns]')
filpanda2 = filpanda.set_index(pd.to_datetime(filpanda.Month))
filpanda2.index

**Violent Crime Numbers Vs Month**

In [ ]:
plt.figure(figsize=(10, 6))

filpanda2.plot(y='CrimeNumbers', x='Month',marker='.', linestyle='-', linewidth=1,\
         subplots=False,color = 'red',
              label='Monthly Crime Numbers',
              title='Violent Crime Numbers Vs Month')

plt.ylabel('Crime Numbers')
plt.xlabel('Year')
plt.show()

**Question 6 : what is the number of crimes per location for a ALL category**

In [ ]:
tot_crimecat = spark.sql("SELECT `Location`, COUNT(*) AS `total crimes` FROM `wmdata` GROUP BY `Location` ORDER BY `total crimes` DESC LIMIT 10")


In [ ]:
tot_crimecat.toPandas()

**Plot Total Crimes by Location**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Assuming you have a Spark DataFrame named "tot_crimecat" with columns "Location" and "total crimes"

# Convert Spark DataFrame to Pandas DataFrame
tot_crimecat_pd = tot_crimecat.toPandas()

# Sort the DataFrame by the total crimes in descending order
tot_crimecat_pd = tot_crimecat_pd.sort_values('total crimes', ascending=False)

# Generate a color palette
color_palette = plt.cm.Set3(np.linspace(0, 1, len(tot_crimecat_pd)))

# Plot the data
plt.figure(figsize=(12, 6))
plt.bar(tot_crimecat_pd['Location'], tot_crimecat_pd['total crimes'], color=color_palette)
plt.xticks(rotation=90)
plt.xlabel('Location')
plt.ylabel('Total Crimes')
plt.title('Total Crimes by Location')
plt.show()


Question 7 : Bar chart showing the top 10 most common types of crimes

In [ ]:
import matplotlib.pyplot as plt
# Convert Spark DataFrame to Pandas DataFrame
wm_top_pd = wm_top.toPandas()

# Extract column values as Python lists
crime_types = wm_top_pd['Crime_type'].tolist()
num_crimes = wm_top_pd['num_crimes'].tolist()

# Create a bar chart of the top 10 crimes
plt.bar(x=crime_types, height=num_crimes,color=color_palette[:10])
plt.xticks(rotation=90)
plt.xlabel('Crime Type')
plt.ylabel('Count')
plt.title('Top 10 Most Common Crimes in West Midlands')
plt.show()

Line chart showing the trend in the number of crimes over time

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Group the data by month and crime type and count the number of crimes
crime_counts = spark.sql("SELECT Month, Crime_Type, COUNT(*) AS count FROM wmdata GROUP BY Month, Crime_Type")

# Pivot the data to create a table of crime counts by month and crime type
pivoted_counts = crime_counts.groupby("Month").pivot("Crime_Type").sum("count").fillna(0)

# Convert the PySpark DataFrame to a Pandas DataFrame and set the index to the month names
pandas_df = pivoted_counts.toPandas().set_index("Month")

# Create a stacked bar chart
months = pandas_df.index
crime_types = pandas_df.columns
totals = pandas_df.sum(axis=1)

plt.figure(figsize=(10, 6))
pandas_df.plot(kind="bar", stacked=True, figsize=(10, 6))
plt.xticks(rotation=90)
plt.xlabel("Month")
plt.ylabel("Number of Crimes")
plt.legend(title="Crime Type", loc="upper left", bbox_to_anchor=(1.0, 1.0))
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA

# Load crime data into a Pandas dataframe
crime_df = spark.sql("SELECT * FROM wmdata").toPandas()

# Convert the "Month" column to datetime type
crime_df['Month'] = pd.to_datetime(crime_df['Month'])

# Group the data by month and count the number of crimes
monthly_crime_counts = crime_df.groupby(pd.Grouper(key='Month', freq='M')).size()

# Create the ARIMA model
model = ARIMA(monthly_crime_counts, order=(2, 2, 1))
model_fit = model.fit()

# Forecast the number of crimes for the next 12 months
forecast = model_fit.forecast(steps=12)

# Plot the historical data and the forecasted values
plt.figure(figsize=(12, 6))
plt.plot(monthly_crime_counts.index, monthly_crime_counts, label='Historical Data')
plt.plot(forecast.index, forecast, label='Forecast')

plt.xlabel('Month')
plt.ylabel('Crime Numbers')
plt.title('Monthly Crime Numbers Forecast')
plt.legend()
plt.show()

